## Getting HMM match states emission probabilities

09/13/2016 - Parsing a pfam .hmm file, and getting the data corresponding to match states emission probabilities.

The data is saved as -log(prob), so later converting it to prob, and saving to file

In [10]:
import fileinput
import sys
import pickle
import pandas as pd
from collections import defaultdict
import numpy as np
import pickle
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>");

In [2]:
curr_dir = !pwd

In [3]:
#Constants
EOF = "//"

In [4]:
#Reading the -log(prob) of the match states emissions from the Pfam HMM file
hmm_log_prob_dict = defaultdict(list)

pfam_hmm_file = fileinput.input(curr_dir[0]+"/zf-C2H2.hmm")
for line in pfam_hmm_file:
    if (line[0:4] == "HMM "):
        aa = line.split()
        aa.remove("HMM")
        print aa
        break

states_cnt = 1
for line in pfam_hmm_file:
    line_list = line.split()
    if (line_list[0] == EOF):
        break
    if (line_list[0] == str(states_cnt)):
        #Saving the probalities as a list for the corresponding HMM state
        prob_strs = line.split()[1:1+len(aa)]
        for i in prob_strs:
            if (i == "*"):
                hmm_log_prob_dict[states_cnt].append(float('inf'))
            else:
                hmm_log_prob_dict[states_cnt].append(float(i))
        states_cnt += 1

fileinput.close()

['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']


In [18]:
hmm_prob_dict_save = hmm_prob_dict.copy()

In [5]:
#Converting to probalities, and saving a vector for each HMM match state
hmm_prob_dict = {}
for state in hmm_log_prob_dict.keys():
    hmm_prob_dict[state] = 1/np.exp(hmm_log_prob_dict[state])
    

In [17]:
hmm_prob_dict

{1: array([ 0.00275905,  0.01895761,  0.02055153,  0.00071609,  0.30503157,
         0.00891589,  0.08290748,  0.02655227,  0.01238821,  0.07284146,
         0.01003255,  0.00587723,  0.00059562,  0.01024946,  0.0008811 ,
         0.01804313,  0.02262477,  0.01879283,  0.00771422,  0.35356781]),
 2: array([ 0.03189296,  0.00785653,  0.02524947,  0.077475  ,  0.01482115,
         0.00155983,  0.01332118,  0.03233387,  0.15632783,  0.01342643,
         0.01577814,  0.02872981,  0.05253425,  0.08649834,  0.11800244,
         0.06556194,  0.11999322,  0.10800584,  0.00017932,  0.03045302]),
 3: array([  3.87399186e-05,   9.99370198e-01,   4.25291855e-05,
          3.87383691e-05,   2.22931746e-05,   1.09054805e-04,
          2.31737589e-05,   1.42402229e-05,   3.19250369e-05,
          3.83172642e-05,   9.59882368e-06,   2.71759002e-05,
          5.79436252e-05,   2.32310689e-05,   4.17897276e-05,
          2.48411253e-05,   2.64972395e-05,   2.01227408e-05,
          1.74894975e-05,   1.7

In [12]:
#Saving to file
with open(curr_dir[0]+'/zinc_match_state_prob.pik', 'wb') as handle:
    pickle.dump(hmm_prob_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)